In [94]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [99]:
current_dir = os.getcwd()

school_file = "schools_il.csv"
tracts_demo_file = "cz_2010_revised_urban_rural.xlsx"
tracts_geo_file = "cb_2018_17_tract_500k.shp"
cz_id_file = "cz_equivalency.xls"

school_file_path = os.path.join(current_dir, 'data', school_file)

geo_dir = os.path.join(current_dir, "data", 'geo')  # directory for geographic data

tracts_demo_path = os.path.join(geo_dir, tracts_demo_file)
tracts_geo_path = os.path.join(geo_dir, tracts_geo_file)
cz_id_path = os.path.join(geo_dir, cz_id_file)

In [100]:
# Load the school data
school_df = pd.read_csv(school_file_path)
# Create an ID GeoDataFrame from the school data
school_gdf = gpd.GeoDataFrame(
school_df,
geometry=gpd.points_from_xy(school_df.lon, school_df.lat),
crs="EPSG:4326"  
)
school_identifier=school_gdf[['geometry']].copy()


In [101]:
# Load the IL tract data
tracts_gdf = gpd.read_file(tracts_geo_path)
tracts_identifier = tracts_gdf[['GEOID', 'geometry']].copy()

In [102]:
def link_school_to_tract(school_identifier, tracts_identifier):
    '''
    This function links schools to tracts by finding the nearest tract to each school.
    Inputs:
    school_file_path: path to the csv file containing school data  
    tracts_geo_path: path to the shapefile containing tract data
    Outputs:
    school_tracts_merged_df: a DataFrame containing the school data with a new column containing the GEOID of the nearest tract
    '''
    tracts_identifier['inside_point'] = tracts_identifier.geometry.representative_point() # always inside the polygon
    # set the geometry to the inside point
    tracts_identifier['geometry_tract'] = tracts_identifier.geometry
    tracts_identifier["geometry"] = tracts_identifier["inside_point"]
    # reproject to the CRS which is suitable for distance calculation and USA
    tracts_identifier = tracts_identifier.to_crs(epsg=5070)
    school_identifier = school_identifier.to_crs(epsg=5070)
    # Find the nearest tract to each school
    school_tracts_id = gpd.sjoin_nearest(school_identifier, tracts_identifier, how="left", distance_col="distance")
    #rename the columns
    school_tracts_id.rename(columns={'GEOID': 'Tract_FIPS'}, inplace=True)
    
    return school_tracts_id


In [103]:
link_school_to_tract(school_identifier, tracts_identifier)

,geometry,index_right,Tract_FIPS,inside_point,geometry_tract,distance
0,POINT (644394.96 2189101.786),1946,17097860809,POINT (-88.11376 42.45769),"POLYGON ((-88.13667 42.47893, -88.13534 42.479...",671.016864
1,POINT (641894.137 2186680.163),898,17097860808,POINT (-88.14937 42.44702),"POLYGON ((-88.18156 42.4388, -88.17731 42.4431...",1158.463532
2,POINT (643655.668 2191339.559),597,17097860806,POINT (-88.11293 42.487),"POLYGON ((-88.13748 42.4798, -88.13453 42.4859...",759.068164
3,POINT (645500.745 2190111.831),1946,17097860809,POINT (-88.11376 42.45769),"POLYGON ((-88.13667 42.47893, -88.13534 42.479...",2146.195988
4,POINT (658087.92 2149052.854),1568,17031803100,POINT (-87.98653 42.09919),"POLYGON ((-87.99251 42.11009, -87.98044 42.109...",894.457781
...,...,...,...,...,...,...
3457,POINT (557213.806 1679796.551),191,17157951100,POINT (-89.64956 38.0031),"POLYGON ((-89.70982 38.01302, -89.70707 38.014...",5584.493471
3458,POINT (557213.806 1679796.551),191,17157951100,POINT (-89.64956 38.0031),"POLYGON ((-89.70982 38.01302, -89.70707 38.014...",5584.493471
3459,POINT (575429.604 1625332.178),2973,17181950200,POINT (-89.37831 37.50218),"POLYGON ((-89.52173 37.56621, -89.52152 37.572...",4382.118449
3460,POINT (603705.768 1677405.521),2228,17055040400,POINT (-89.0853 37.90888),"POLYGON ((-89.15103 37.88758, -89.15077 37.906...",3544.279895


In [104]:
# Load the tracts demographic data
tracts_df = pd.read_excel(tracts_demo_path, header=1)
tracts_df.rename(columns={'State-County FIPS Code': 'FIPS'}, inplace=True)
tracts_fips = tracts_df[['FIPS','Select State','State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)']].copy()

In [105]:
# Load the CZ ID data
cz_id_df = pd.read_excel(cz_id_path, header=0)
cz_id = cz_id_df[['Commuting Zone ID, 1990', 'FIPS']].copy()

In [106]:
def link_tract_to_cz(tracts_fips, cz_id):
    # merge the tracts and commuting zones
    tracts_cz_merged_df = pd.merge(tracts_fips, cz_id, on='FIPS', how='left')
    #keep the tracts in IL, IN, WI, MI
    tracts_cz_merged_df = tracts_cz_merged_df[
        (tracts_cz_merged_df['Select State'] == 'IL') |
        (tracts_cz_merged_df['Select State'] == 'IN') |
        (tracts_cz_merged_df['Select State'] == 'WI') |
        (tracts_cz_merged_df['Select State'] == 'MI')
    ]
    #convert the float type to int
    tracts_cz_merged_df['Commuting Zone ID, 1990'] = tracts_cz_merged_df['Commuting Zone ID, 1990'].astype(int)
    new_order = ['Select State','Commuting Zone ID, 1990', 'FIPS', 'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)']
    tracts_cz_merged_df = tracts_cz_merged_df.reindex(columns=new_order)
    #rename the columns
    tracts_cz_merged_df.rename(columns={'Select State': 'State', 'Commuting Zone ID, 1990': 'CZ_ID', 'FIPS': 'County_FIPS', 'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)': 'Tract_FIPS'}, inplace=True)

    return tracts_cz_merged_df

In [107]:
link_tract_to_cz(tracts_fips, cz_id)

,State,CZ_ID,County_FIPS,Tract_FIPS
20959,IL,25000,17001,17001000100
20960,IL,25000,17001,17001000201
20961,IL,25000,17001,17001000202
20962,IL,25000,17001,17001000400
20963,IL,25000,17001,17001000500
...,...,...,...,...
72920,WI,22700,55141,55141011300
72921,WI,22700,55141,55141011400
72922,WI,22700,55141,55141011500
72923,WI,22700,55141,55141011600
